In [1]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPUfrom sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

23/02/09 19:30:46 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/02/09 19:30:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0cd9957b-cfc7-438f-a4ab-205d0c0a266a;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

23/02/09 19:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

In [3]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [4]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("text_train.csv")
trainDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import col
trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      DA|   24|
|      SN|   20|
|      LR|   14|
|      VC|   13|
|      AR|   13|
|      NA|   12|
|      SD|   12|
|     NAV|   12|
|      CC|   11|
+--------+-----+



In [6]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      DA|   24|
|      SN|   20|
|      LR|   14|
|      VC|   13|
|      AR|   13|
|      NA|   12|
|      SD|   12|
|     NAV|   12|
|      CC|   11|
+--------+-----+



In [7]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(50)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('classifer_logs')\

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

use_pipelineModel = use_clf_pipeline.fit(trainDataset)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.
[ \ ]

2023-02-09 19:31:02.717369: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

[OK!]


2023-02-09 19:31:11.398590: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/b2b542bf6de5_classifier_dl7975566201141353444
2023-02-09 19:31:11.426880: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2023-02-09 19:31:11.426912: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/b2b542bf6de5_classifier_dl7975566201141353444
2023-02-09 19:31:11.763292: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-02-09 19:31:12.221476: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/b2b542bf6de5_classifier_dl7975566201141353444
2023-02-09 19:31:12.344477: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 945900 microsecond

Training started - epochs: 50 - learning_rate: 0.003 - batch_size: 32 - training_examples: 105 - classes: 9
Epoch 1/50 - 0.24s - loss: 8.683384 - acc: 0.26736113 - batches: 4
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.19s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           7	 7	 1	 0.5	 0.875	 0.6363636
NA           0	 0	 1	 0.0	 0.0	 0.0
SD           0	 0	 5	 0.0	 0.0	 0.0
VC           0	 0	 2	 0.0	 0.0	 0.0
CC           0	 0	 1	 0.0	 0.0	 0.0
AR           0	 0	 1	 0.0	 0.0	 0.0
SN           5	 4	 0	 0.5555556	 1.0	 0.71428573
LR           3	 0	 0	 1.0	 1.0	 1.0
tp: 15 fp: 11 fn: 11 labels: 8
Macro-average	 prec: 0.25694445, rec: 0.359375, f1: 0.2996479
Micro-average	 prec: 0.5769231, recall: 0.5769231, f1: 0.5769231
Epoch 2/50 - 0.02s - loss: 7.5659637 - acc: 0.5127315 - batches: 4
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           7	 4	 

Epoch 16/50 - 0.01s - loss: 5.8326664 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           5	 0	 3	 1.0	 0.625	 0.7692308
NAV          0	 2	 0	 0.0	 0.0	 0.0
NA           1	 0	 0	 1.0	 1.0	 1.0
SD           0	 0	 5	 0.0	 0.0	 0.0
VC           2	 6	 0	 0.25	 1.0	 0.4
CC           1	 0	 0	 1.0	 1.0	 1.0
AR           0	 0	 1	 0.0	 0.0	 0.0
SN           5	 1	 0	 0.8333333	 1.0	 0.90909094
LR           3	 0	 0	 1.0	 1.0	 1.0
tp: 17 fp: 9 fn: 9 labels: 9
Macro-average	 prec: 0.5648148, rec: 0.625, f1: 0.5933852
Micro-average	 prec: 0.65384614, recall: 0.65384614, f1: 0.65384614
Epoch 17/50 - 0.01s - loss: 5.8082495 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           5	 0	 3	 1.0	 0.625	 0.7692308
NAV          0	 2	 0	 0.0	 0.0	 0.0
NA           1	 0	 0	 1

Epoch 32/50 - 0.01s - loss: 5.662019 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           5	 0	 3	 1.0	 0.625	 0.7692308
NAV          0	 2	 0	 0.0	 0.0	 0.0
NA           1	 2	 0	 0.33333334	 1.0	 0.5
SD           0	 0	 5	 0.0	 0.0	 0.0
VC           2	 4	 0	 0.33333334	 1.0	 0.5
CC           1	 0	 0	 1.0	 1.0	 1.0
AR           0	 0	 1	 0.0	 0.0	 0.0
SN           5	 1	 0	 0.8333333	 1.0	 0.90909094
LR           3	 0	 0	 1.0	 1.0	 1.0
tp: 17 fp: 9 fn: 9 labels: 9
Macro-average	 prec: 0.5, rec: 0.625, f1: 0.5555556
Micro-average	 prec: 0.65384614, recall: 0.65384614, f1: 0.65384614
Epoch 33/50 - 0.01s - loss: 5.6578207 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           5	 0	 3	 1.0	 0.625	 0.7692308
NAV          0	 2	 0	 0.0	 0.0	 0.0
NA           1	 2

In [8]:
use_pipelineModel.stages[2].write().overwrite().save('Text_Classification')

In [9]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
loaded_ner_model = ClassifierDLModel.load("Text_Classification")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        loaded_ner_model
    ])

#use_pipelineModel = use_clf_pipeline.fit(trainDataset)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [10]:
from sparknlp.base import LightPipeline
#clf_pipelineModel = use_clf_pipeline.fit(trainDataset)

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = use_clf_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['class'])

{'document': ['navigate to kitchen'], 'sentence_embeddings': ['navigate to kitchen'], 'class': ['NAV']}
['NAV']


In [11]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['class'])
    text = input("Enter New Text\n")

Enter Testing Text
take me home
['NAV']
Enter New Text
where am I
['LR']
Enter New Text
am I in the kitchen
['LR']
Enter New Text
is this the bathroom
['LR']
Enter New Text
what can I do here
['AR']
Enter New Text
brush teeth
['DA']
Enter New Text
open camera
['CC']
Enter New Text
close camera
['CC']
Enter New Text
camera
['CC']
Enter New Text
video
['VC']
Enter New Text
rewind
['VC']
Enter New Text
stop
['VC']
Enter New Text
pause
['VC']
Enter New Text
close
['SN']
Enter New Text
shutdown
['VC']
Enter New Text
shut 
['AR']
Enter New Text
down
['AR']
Enter New Text
power down
['VC']
Enter New Text
close video
['VC']
Enter New Text
kiss gwen
['NA']
Enter New Text
smoke weed
['DA']
Enter New Text
Hi how is your new job
['NA']
Enter New Text
what are you doing this afternoon
['NAV']
Enter New Text
make potroast
['DA']
Enter New Text
clean dishes
['DA']
Enter New Text
I want to clean the dishes
['DA']
Enter New Text
How do I clean the oven
['DA']
Enter New Text
exit
